In [2]:
import pandas as pd
import random
import os, io, json
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms

# Parse original json file

In [2]:
images_dir = "../datasets/AVA/images"
captions_file_cleaned = "../datasets/CLEAN_AVA_FULL_COMMENTS.json"

In [6]:
with io.open(captions_file_cleaned, 'r', encoding = 'utf-8') as f:
    json_file = json.load(f)

In [7]:
df_raw = pd.DataFrame(json_file['images'])

In [8]:
df_raw.head()

,sentids,filepath,filename,imgid,split,sentences,cocoid
0,,,953958.jpg,953958,train,"[{'bigrams': [[['geometry', 'NN'], ['man', 'NN...",953958
1,,,954184.jpg,954184,train,"[{'bigrams': [[['puffed', 'NN'], ['king', 'NN'...",954184
2,,,954113.jpg,954113,train,"[{'bigrams': [[['great', 'JJ'], ['decision', '...",954113
3,,,953980.jpg,953980,train,"[{'bigrams': [[['deep', 'JJR'], ['dof', 'NN']]...",953980
4,,,953349.jpg,953349,train,"[{'bigrams': [[['nice', 'JJ'], ['nice', 'JJ']]...",953349


In [ ]:
df_raw['sentences'][1000][0]

In [13]:
filenames_list = []
captions_list = []
split_list = []

for i, img in enumerate(df_raw['filename']):
    for j, caption in enumerate(df_raw['sentences'][i]):
        if os.path.exists(os.path.join(images_dir, img)):
            filenames_list.append(img)
            captions_list.append(caption['clean'])
            split_list.append(df_raw['split'][i])

In [15]:
df_full_dict = {'filename': filenames_list, 'clean_sentence': captions_list}
df_full = pd.DataFrame(df_full_dict)

In [16]:
df_full.head()

,filename,clean_sentence,split
0,953417.jpg,nice you have mad macro skills you'll notice i...,train
1,953417.jpg,frog with attitude love it reminds me of jabba...,train
2,953417.jpg,he does look kind of lonely funky colors,train
3,953417.jpg,no thanks he is quite blue is'nt he i like this,train
4,953417.jpg,great shot,train


In [59]:
df_full.to_feather('../datasets/AVA/CLEAN_AVA_SAMPLE_COMMENTS.feather')

# Get 10% sample of data

In [3]:
captions_file = "../datasets/AVA/CLEAN_AVA_FULL_COMMENTS.feather"
# 3,330,881 comments total
# 253,895 unique images

In [4]:
df = pd.read_feather(captions_file)

In [5]:
df.head(15)

,filename,clean_sentence,split
0,953417.jpg,nice you have mad macro skills you'll notice i...,train
1,953417.jpg,frog with attitude love it reminds me of jabba...,train
2,953417.jpg,he does look kind of lonely funky colors,train
3,953417.jpg,no thanks he is quite blue is'nt he i like this,train
4,953417.jpg,great shot,train
5,953417.jpg,very cute,train
6,953417.jpg,funny shot very nice perspective and lovely co...,train
7,953777.jpg,well done i love the perspective,train
8,953777.jpg,nice placement i see this made,train
9,953777.jpg,on your profile page i suspect you will be rot...,train


In [6]:
print(len(df['filename']))
print(len(pd.unique(df['filename'])))

3330881
253895


In [7]:
#df_filtered = df.drop(df[df['clean_sentence'].map(lambda x: len(x.split()) < 4)].index)
df_filtered = df

In [8]:
print(len(df_filtered['filename']))
print(len(pd.unique(df_filtered['filename'])))

3330881
253895


In [9]:
df_filtered.head(15)

,filename,clean_sentence,split
0,953417.jpg,nice you have mad macro skills you'll notice i...,train
1,953417.jpg,frog with attitude love it reminds me of jabba...,train
2,953417.jpg,he does look kind of lonely funky colors,train
3,953417.jpg,no thanks he is quite blue is'nt he i like this,train
4,953417.jpg,great shot,train
5,953417.jpg,very cute,train
6,953417.jpg,funny shot very nice perspective and lovely co...,train
7,953777.jpg,well done i love the perspective,train
8,953777.jpg,nice placement i see this made,train
9,953777.jpg,on your profile page i suspect you will be rot...,train


In [10]:
filenames_list = pd.unique(df_filtered['filename']).tolist()

In [11]:
filename_sample = random.sample(filenames_list, int(len(filenames_list)*0.1))

In [12]:
df_sampled = df_filtered.drop(df_filtered[~df_filtered['filename'].isin(filename_sample)].index)

In [13]:
df_sampled = df_sampled.drop(columns='split').reset_index(drop=True)

In [14]:
df_sampled.head(15)

,filename,clean_sentence
0,953756.jpg,good ambiance and pp it looks like where i gre...
1,953756.jpg,love the tree in the middle
2,953756.jpg,gritty desat image wonderful
3,953756.jpg,i like the grainy effet in this image gives it...
4,953756.jpg,i like the use of grain throughout
5,953550.jpg,this photo is absolutely stunning
6,953550.jpg,very nicely done nice job getting rid of the e...
7,953550.jpg,congrats on it all
8,953550.jpg,wow what a view
9,953550.jpg,lucky lucky you good use of the fish eye


In [15]:
print(len(df_sampled))
print(len(pd.unique(df_sampled['filename'])))

334185
25389


In [16]:
df_sampled.to_csv('../datasets/AVA/AVA_sample_10percent.txt', index=False)
df_sampled.to_feather('../datasets/AVA/AVA_sample_10percent.feather')

# Creating file for gpt2 language model

In [5]:
images_dir = "../datasets/AVA/images"
captions_file_cleaned_subjective = "../datasets/AVA/CLEAN_AVA_FULL_AFTER_SUBJECTIVE_CLEANING.json"

In [6]:
with io.open(captions_file_cleaned_subjective, 'r', encoding = 'utf-8') as f:
    json_file = json.load(f)

In [7]:
df_raw = pd.DataFrame(json_file['images'])
df_raw.head()

,sentids,filepath,filename,imgid,split,sentences,cocoid
0,,,953958.jpg,953958,train,"[{'bigrams': [[['geometry', 'NN'], ['man', 'NN...",953958
1,,,954184.jpg,954184,train,"[{'bigrams': [[['puffed', 'NN'], ['king', 'NN'...",954184
2,,,954113.jpg,954113,train,"[{'bigrams': [[['great', 'JJ'], ['decision', '...",954113
3,,,953980.jpg,953980,train,"[{'bigrams': [[['deep', 'JJR'], ['dof', 'NN']]...",953980
4,,,953349.jpg,953349,train,"[{'bigrams': [[['nice', 'JJ'], ['nice', 'JJ']]...",953349


In [8]:
len(df_raw)

240060

In [29]:
df_raw['sentences'][0][0]['clean']

'more geometry than any man has a right to.'

In [20]:
filenames_list = []
captions_list = []

for i, img in enumerate(df_raw['filename']):
    for j, caption in enumerate(df_raw['sentences'][i]):
        if os.path.exists(os.path.join(images_dir, img)):
            filenames_list.append(img)
            captions_list.append(caption['clean'])

In [28]:
captions_list[10]

"you have the widest armspan i think i've ever seen. hopefully that's a bit of a factor of the wide lens angle!"

In [22]:
df_full_dict = {'filename': filenames_list, 'clean_sentence': captions_list}
df_full = pd.DataFrame(df_full_dict)
df_full.head()

,filename,clean_sentence
0,953417.jpg,. nice! you have mad macro skills! you'll noti...
1,953417.jpg,funny shot. very nice perspective and lovely c...
2,953777.jpg,on your profile page. i suspect you will be ro...
3,953777.jpg,awesome picture! great angle and lighting and ...
4,953777.jpg,that's really within meters of your house? if ...


In [42]:
df_full.to_feather('../datasets/AVA/AVA-captions_full.feather')

In [48]:
with open('../datasets/AVA/AVA-captions_full_text.txt', 'w', encoding="utf-8") as f:
    f.write(" ".join(captions_list))